In [77]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def main():
    # Training settings
    


    device = torch.device("cuda")

    train_kwargs = {'batch_size': 32}
    test_kwargs = {'batch_size': 32}

    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=0.01)

    for epoch in range(1, 20):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        torch.save(model.state_dict(), "mnist_cnn.pt")


main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.318118
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.713731
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.758172
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.569233
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.734471
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.654326
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.514348
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.659032
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.465185
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.445422
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.213862
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.368433
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.799168
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.435594
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.516805
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.414903
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.283805
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.186711
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.383691

Test set: Average loss: 0.2505, Accuracy: 9264/10000 (93%)